In [1]:
P.<x,y,z> = ProjectiveSpace(QQ,2)
f = DynamicalSystem_projective([x^2-y^2, x^2-z^2, y^2-z^2])
f.affine_preperiodic_model(1)

Dynamical System of Projective Space of dimension 2 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^2 - y^2 : x^2 - z^2 : y^2 - z^2)

In [3]:
f.periodic_points(1, return_scheme=True).rational_points(F=QQbar)

[(-1 : -1 : 1),
 (-1 : 0 : 1),
 (-1 : 1 : 1),
 (-0.50000000000000000? - 0.866025403784439?*I : 0.50000000000000000? - 0.866025403784439?*I : 1),
 (-0.50000000000000000? + 0.866025403784439?*I : 0.50000000000000000? + 0.866025403784439?*I : 1),
 (1 : -1 : 1),
 (1 : 1 : 1)]

In [4]:
P.<x,y,z> = ProjectiveSpace(QQ,2)
f = DynamicalSystem_projective([x^2, y^2, z^2])
g = f.affine_preperiodic_model(1)
g.change_ring(QQbar).periodic_points(1)

[(-1 : 1 : 1),
 (-1/2 : 1/2 : 1),
 (-1/2 : 1 : 1),
 (-1/3 : 2/3 : 1),
 (0 : 0 : 1),
 (0 : 1/2 : 1),
 (0 : 1 : 1)]

In [2]:
P.<x,y,z> = ProjectiveSpace(QQ, 2)
f = DynamicalSystem_projective([x^2, y^2, z^2])
f.affine_preperiodic_model(1, 2)

Dynamical System of Projective Space of dimension 2 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (4/5*x^2 : 6/5*x^2 - 4/5*x*y + 2/5*y^2 : 6/25*x^2 - 4/25*x*y + 3/25*y^2 - 2/5*y*z + z^2)

In [1]:
P.<x,y,z> = ProjectiveSpace(GF(3), 2)
f = DynamicalSystem_projective([x^2, y^2, z^2])
f.affine_preperiodic_model(1)
f.periodic_points(1)

[(0 : 0 : 1),
 (0 : 1 : 0),
 (0 : 1 : 1),
 (1 : 0 : 0),
 (1 : 0 : 1),
 (1 : 1 : 0),
 (1 : 1 : 1)]

In [1]:
def affine_preperiodic_model(f, m, n):
    from itertools import product, count
    from sage.rings.polynomial.polynomial_ring import is_PolynomialRing
    from sage.rings.polynomial.multi_polynomial_ring_base import is_MPolynomialRing
    from sage.rings.fraction_field import is_FractionField
    CR = f.coordinate_ring()
    dom = f.domain()
    PS = f.codomain().ambient_space()
    N = PS.dimension_relative() + 1
    R = f.base_ring()
    F_1 = f.nth_iterate_map(n+m)
    F_2 = f.nth_iterate_map(m)
    L = [F_1[i]*F_2[j] - F_1[j]*F_2[i] for i in range(N)
        for j in range(i+1, N)]
    X = PS.subscheme(L + list(dom.defining_polynomials()))
    hyperplane_at_infinity = PS.subscheme(CR.gens()[-1])
    hyperplane_found = False
    attempted_combinations = {}
    
    if R.is_field():
        F = R
    else:
        F = FractionField(R)
    
    if X.intersection(hyperplane_at_infinity).change_ring(F).dimension() >= 0:
        # in order to find a hyperplane to move to infinity, 
        if R.is_finite():
            for tup in product(R, repeat=N):
                if list(tup) != [0]*N:
                    if PS(tup) not in attempted_combinations:
                        hyperplane = PS.subscheme(sum([tup[i]*PS.gens()[i] for i in range(N)]))
                        if X.intersection(hyperplane).change_ring(F).dimension() < 0:
                            hyperplane_found = True
                            print(hyperplane)
                            break
            if not hyperplane_found:
                raise ValueError('no possible conjugation over %s makes all preperiodic points affine' %R)
        else:
            # if the characteristic is 0, R contains Z
            if R.characteristic() == 0:
                for height_bound in count(1):
                    terms = ZZ.range(height_bound)
                    for tup in product(terms, repeat=N):
                        if list(tup) != [0]*N:
                            if PS(tup) not in attempted_combinations:
                                hyperplane = PS.subscheme(sum([tup[i]*PS.gens()[i] for i in range(N)]))
                                if X.intersection(hyperplane).change_ring(F).dimension() < 0:
                                    hyperplane_found = True
                                    print(hyperplane)
                                    break
                    if hyperplane_found:
                        break
            else:
                if is_PolynomialRing(R) or is_MPolynomialRing(R) or is_FractionField(R):
                    var = R.gen()
                    for degree in count(0):
                        ZZ_terms = ZZ.range(R.characteristic())
                        terms = ZZ_terms[:]
                        for i in ZZ_terms:
                            terms.append(i*var**degree)
                        for tup in product(terms, repeat=N):
                            if list(tup) != [0]*N:
                                if PS(tup) not in attempted_combinations:
                                    print(tup)
                                    hyperplane = PS.subscheme(sum([tup[i]*PS.gens()[i] for i in range(N)]))
                                    if X.intersection(hyperplane).change_ring(F).dimension() < 0:
                                        hyperplane_found = True
                                        break
                        if hyperplane_found:
                            break
                else:
                    raise ValueError('cannot find affine periodic model over %s' %(R))
        source = PS.subscheme(CR.gens()[-1])
        mat = PS.hyperplane_transformation_matrix(source, hyperplane)
        if R.is_field():
            return f.conjugate(mat)
        return f.conjugate(mat, adjugate=True)
    return f

In [2]:
P.<x,y,z> = ProjectiveSpace(QQ,2)
f = DynamicalSystem_projective([x^2, y^2, z^2])
f.affine_preperiodic_model(1)

Closed subscheme of Projective Space of dimension 2 over Rational Field defined by:
  x + y + z


Dynamical System of Projective Space of dimension 2 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (-x^2 : 2*x^2 + 2*x*y + y^2 : 2*x^2 + 2*x*y + 2*y^2 - 2*y*z + z^2)

In [4]:
P.<x,y,z> = ProjectiveSpace(GF(9),2)
f = DynamicalSystem_projective([x^2, y^2, z^2])
f.affine_preperiodic_model(1)

Dynamical System of Projective Space of dimension 2 over Finite Field in z2 of size 3^2
  Defn: Defined on coordinates by sending (x : y : z) to
        ((z2 + 1)*x^2 : (z2 + 1)*x^2 + (z2 + 1)*x*y + (-z2 - 1)*y^2 : (z2 - 1)*x^2 + (z2 - 1)*x*y - y^2 + (-z2)*y*z + z^2)

In [28]:
sage: R.<c> = GF(3)[]
            sage: P.<x,y,z> = ProjectiveSpace(FractionField(R), 2)
            sage: f = DynamicalSystem_projective([x^2, y^2, z^2])
            sage: f.affine_preperiodic_model(1)

Dynamical System of Projective Space of dimension 2 over Fraction Field of Univariate Polynomial Ring in c over Finite Field of size 3
  Defn: Defined on coordinates by sending (x : y : z) to
        ((c/(c + 2))*x^2 : (c/(c + 2))*x^2 + (c/(c + 2))*x*y + (2*c/(c + 2))*y^2 : (2*c/(c^2 + c + 1))*x^2 + (2*c/(c^2 + c + 1))*x*y + ((c + 1)/(c^2 + c + 1))*y^2 + (2/(c + 2))*y*z + z^2)

In [31]:
K.<k> = CyclotomicField(3)
P.<x,y,z> = ProjectiveSpace(K, 2)
f = DynamicalSystem_projective([x^2 + k*x*y + y^2, z^2, y^2])
f.affine_preperiodic_model(1, 1)

Closed subscheme of Projective Space of dimension 2 over Cyclotomic Field of order 3 and degree 2 defined by:
  x


Dynamical System of Projective Space of dimension 2 over Cyclotomic Field of order 3 and degree 2
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^2 + y^2 + (-k + 2)*x*z - 2*y*z + (-k + 3)*z^2 : -2*x^2 + (k - 4)*x*z + (k - 3)*z^2 : -x^2 + (k - 2)*x*z + (k - 2)*z^2)

In [5]:
P.<x,y> = ProjectiveSpace(QQ, 1)
f = DynamicalSystem_projective([x^2 + y^2, y^2])
g, mat = f.affine_preperiodic_model(1, return_conjugation=True)
g == f.conjugate(mat)

True

In [2]:
R.<c> = QQ[]
P.<x,y> = ProjectiveSpace(R, 1)
f = DynamicalSystem_projective([x^2 + 2*y^2, y^2])
f.affine_preperiodic_model(1)

Dynamical System of Projective Space of dimension 1 over Univariate Polynomial Ring in c over Rational Field
  Defn: Defined on coordinates by sending (x : y) to
        (3*x^2 + 6*x*y + 4*y^2 : (-2)*x^2 + (-4)*x*y + (-3)*y^2)

In [12]:
%%time
R.<c> = GF(3)[]
P.<x,y,z> = ProjectiveSpace(R, 2)
f = DynamicalSystem_projective([x^2, y^2, z^2])
f.affine_preperiodic_model(0,1)

CPU times: user 1.31 s, sys: 222 ms, total: 1.54 s
Wall time: 2.06 s


Dynamical System of Projective Space of dimension 2 over Univariate Polynomial Ring in c over Finite Field of size 3
  Defn: Defined on coordinates by sending (x : y : z) to
        ((2*c^4 + c^3)*x^2 : (2*c^4 + c^3)*x^2 + (2*c^4 + c^3)*x*y + (c^4 + 2*c^3)*y^2 : c^3*x^2 + c^3*x*y + (2*c^3 + 2*c^2)*y^2 + (c^3 + 2*c^2)*y*z + (2*c^4 + 2*c^3 + 2*c^2)*z^2)

In [11]:
P.<x,y,z> = ProjectiveSpace(QQ, 2)
X = P.subscheme(2*y - z)
f = DynamicalSystem_projective([x^2 + y^2, z^2 + y^2, z^2], domain=X)
g = f.affine_preperiodic_model(0, 1)
g.periodic_points(1, return_scheme=True).intersection(P.subscheme(z)).dimension()

-1

In [14]:
R.<c> = GF(3)[]
P.<x,y> = ProjectiveSpace(R, 1)
f = DynamicalSystem_projective([x^2 +c*y^2, y^2])
f.affine_preperiodic_model(0, 1)

Dynamical System of Projective Space of dimension 1 over Univariate Polynomial Ring in c over Finite Field of size 3
  Defn: Defined on coordinates by sending (x : y) to
        ((c + 1)*x^2 + (2*c + 2)*x*y + (c + 2)*y^2 : 2*c*x^2 + c*x*y + (2*c + 2)*y^2)

In [20]:
P.<x,y> = ProjectiveSpace(QQ, 1)
f = DynamicalSystem_projective([x^2 + 2*y^2, x^2])
f.affine_preperiodic_model(0, 1, return_conjugation=True)

(
Dynamical System of Projective Space of dimension 1 over Rational Field
  Defn: Defined on coordinates by sending (x : y) to                   
        (x^2 + 2*y^2 : x^2)                                            ,

[1 0 0]
[0 1 0]
[0 0 1]
)